In [1]:
import re
import time
import random
import pandas as pd
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse, parse_qs

BASE = "https://www.scb.se"
UPDATE_ENDPOINT = "https://www.scb.se/DokumentationSammanstallning/UpdateAmnesomrade"

# ---- dina mål ----
KEEP_YEARS = {2024, 2025}
KEEP_TYPES = {"KD", "StaF"}

HEADERS = {
    "User-Agent": "Mozilla/5.0",
    "Accept-Language": "sv-SE,sv;q=0.9,en;q=0.8",
    # bra att ha referer mot sidan som använder dropdownen
    "Referer": "https://www.scb.se/dokumentation/kvalitet-och-framtagning/",
}

# Snäll belastning
SLEEP_MIN = 0.15
SLEEP_MAX = 0.5
TIMEOUT = 60


# -------------------------
# Hjälpfunktioner
# -------------------------
def classify_type(u: str) -> str:
    s = (u or "").lower()
    if "_kd_" in s:
        return "KD"
    if "_staf_" in s or "framstallning" in s:
        return "StaF"
    if "_bs_" in s:
        return "BaS"
    return "Övrigt"


def extract_product_code(filename: str):
    m = re.match(r"^([a-z]{2}\d{4})", (filename or "").lower())
    return m.group(1) if m else pd.NA


def extract_ref_year(filename: str):
    """
    Tar första riktiga året (19xx/20xx) i filnamnet, så am0501 inte blir 0501.
    Ex: am0501_kd_2021_20221207.pdf -> [2021, 2022] -> 2021
    """
    years = re.findall(r"(?:19|20)\d{2}", str(filename))
    return int(years[0]) if years else pd.NA


def extract_amnesomrade_id_from_url(url: str) -> str | None:
    q = parse_qs(urlparse(url).query)
    v = q.get("amnesomrade", [None])[0]
    return v


def fetch_amnesomrade_html(session: requests.Session, amnes_id: str) -> str:
    r = session.get(
        UPDATE_ENDPOINT,
        params={"amnesomrade": amnes_id},
        headers=HEADERS,
        timeout=TIMEOUT,
    )
    r.raise_for_status()
    return r.text


def extract_pdf_links(html: str, base_url: str):
    soup = BeautifulSoup(html, "html.parser")
    rows = []
    for a in soup.select("a[href]"):
        href = (a.get("href") or "").strip()
        if not href:
            continue
        if ".pdf" in href.lower():
            full = urljoin(base_url, href)
            linktext = a.get_text(" ", strip=True)
            fil = full.split("/")[-1].split("?")[0]
            rows.append((linktext, full, fil))
    return rows


# -------------------------
# 1) Mata in dina ämnesområdes-URL:er
# -------------------------
amnes_urls = [
    "https://www.scb.se/DokumentationSammanstallning/UpdateAmnesomrade?amnesomrade=55707",
    "https://www.scb.se/DokumentationSammanstallning/UpdateAmnesomrade?amnesomrade=55707",
    "https://www.scb.se/DokumentationSammanstallning/UpdateAmnesomrade?amnesomrade=79826",
    "https://www.scb.se/DokumentationSammanstallning/UpdateAmnesomrade?amnesomrade=57291",
    "https://www.scb.se/DokumentationSammanstallning/UpdateAmnesomrade?amnesomrade=60584",
    "https://www.scb.se/DokumentationSammanstallning/UpdateAmnesomrade?amnesomrade=57444",
    "https://www.scb.se/DokumentationSammanstallning/UpdateAmnesomrade?amnesomrade=57780",
    "https://www.scb.se/DokumentationSammanstallning/UpdateAmnesomrade?amnesomrade=57780",
    "https://www.scb.se/DokumentationSammanstallning/UpdateAmnesomrade?amnesomrade=196347",
    "https://www.scb.se/DokumentationSammanstallning/UpdateAmnesomrade?amnesomrade=56199",
    "https://www.scb.se/DokumentationSammanstallning/UpdateAmnesomrade?amnesomrade=57979",
    "https://www.scb.se/DokumentationSammanstallning/UpdateAmnesomrade?amnesomrade=57979",
    "https://www.scb.se/DokumentationSammanstallning/UpdateAmnesomrade?amnesomrade=57979",
    # + lägg till fler vid behov
]

# plocka id:n och deduplicera
amnes_ids = sorted({extract_amnesomrade_id_from_url(u) for u in amnes_urls if extract_amnesomrade_id_from_url(u)})
print("Antal unika ämnesområdes-id:", len(amnes_ids), amnes_ids)


# -------------------------
# 2) Loop över alla ämnesområden och bygg DataFrame
# -------------------------
session = requests.Session()
all_rows = []

for amnes_id in amnes_ids:
    try:
        html = fetch_amnesomrade_html(session, amnes_id)
        pdfs = extract_pdf_links(html, BASE)

        for linktext, url, fil in pdfs:
            all_rows.append({
                "amnesomrade_id": amnes_id,
                "linktext": linktext,
                "url": url,
                "fil": fil,
            })

        time.sleep(random.uniform(SLEEP_MIN, SLEEP_MAX))

    except Exception as e:
        # logga men fortsätt
        all_rows.append({
            "amnesomrade_id": amnes_id,
            "linktext": pd.NA,
            "url": pd.NA,
            "fil": pd.NA,
            "error": str(e)
        })

df = pd.DataFrame(all_rows)

# rensa bort rader utan url (t.ex. logg-felrader)
df = df[df["url"].notna()].copy()

# -------------------------
# 3) Lägg på typ / produktkod / år och filtrera 2024/2025
# -------------------------
df["typ"] = df["url"].apply(classify_type)
df["produktkod"] = df["fil"].apply(extract_product_code)
df["år"] = df["fil"].apply(extract_ref_year)

df_filtered = df[
    df["typ"].isin(KEEP_TYPES) &
    df["år"].isin(KEEP_YEARS)
].copy()

# -------------------------
# 4) Spara
# -------------------------
df.to_csv("scb_lankar_alla_amnesomraden.csv", index=False, encoding="utf-8")
df_filtered.to_csv("scb_kd_staf_2024_2025_alla_amnesomraden.csv", index=False, encoding="utf-8")

print("Totalt antal pdf-länkar:", len(df))
print("KD+StaF 2024/2025:", len(df_filtered))

Antal unika ämnesområdes-id: 9 ['196347', '55707', '56199', '57291', '57444', '57780', '57979', '60584', '79826']
Totalt antal pdf-länkar: 3472
KD+StaF 2024/2025: 194


NameError: name 'df_filtered' is not defined